# 1. Trajectory Simulation

In [1]:
%matplotlib inline
import numpy as np
import tables
import matplotlib.pyplot as plt
import seaborn as sns
import pybromo as pbm
import phconvert as phc
import pandas as pd
import fretbursts as fb
import burstH2MM as bhm
print('Numpy version:', np.__version__)
print('PyTables version:', tables.__version__)
print('PyBroMo version:', pbm.__version__)
print('phconvert version:', phc.__version__)


def patch_asscalar(a):
    return a.item()

setattr(np, "asscalar", patch_asscalar)

 - Optimized (cython) burst search loaded.
--------------------------------------------------------------
 You are running FRETBursts (version 0.8.3).

 If you use this software please cite the following paper:

   FRETBursts: An Open Source Toolkit for Analysis of Freely-Diffusing Single-Molecule FRET
   Ingargiola et al. (2016). http://dx.doi.org/10.1371/journal.pone.0160716 

--------------------------------------------------------------
Numpy version: 1.26.4
PyTables version: 3.9.2
PyBroMo version: 0.8.1
phconvert version: 0.9.1


In [7]:
# Code from https://github.com/OpenSMFS/PyBroMo/blob/master/notebooks/PyBroMo%20-%201.%20Simulate%203D%20trajectories%20-%20single%20core.ipynb
# Adapted to fit my case
# Make diffusion trajectory

rs = np.random.RandomState()
print('Initial random state:', pbm.hashfunc(rs.get_state()))

# Simulation time step (seconds)
t_step = 0.5e-6

# Time duration of the simulation (seconds)
t_max = 500

# Diffusion coefficient
Du = 10.0            # um^2 / s
D1 = Du*(1e-6)**2    # m^2 / s

# Simulation box definition
box = pbm.Box(x1=-3e-6, x2=3e-6, y1=-3e-6, y2=3e-6, z1=-4E-6, z2=4E-6)

# Particles definition
n_particles = 15
P = pbm.Particles.from_specs(
    num_particles=(n_particles,),
    D=(D1,),
    box=box, rs=rs)

# PSF definition
psf = pbm.NumericPSF()


# Particle simulation definition
S = pbm.ParticlesSimulation(t_step=t_step, t_max=t_max, 
                            particles=P, box=box, psf=psf)

print('Current random state:', pbm.hashfunc(rs.get_state()))
print(S)
S.print_sizes()

Initial random state: 185fb38422d4ea48d9def26fdb1e8d2778de2af2
Current random state: c6bb2010859607a0e526550fa72a74731a01cab9
Box: X 6.0um, Y 6.0um, Z 8.0um
#Particles: 15 D: 1e-11, 86.5 pM, t_step 0.5us, t_max 500.0s ID_EID 0 0
  Number of particles: 15
  Number of time steps: 1000000000
  Emission array - 1 particle (float32): 3814.7 MB
  Emission array (float32): 57220.5 MB
  Position array (float32): 171661.4 MB 


In [8]:
S.simulate_diffusion(
    total_emission=False,
    save_pos=False,
    verbose=True,
    rs=rs,
    chunksize=2**23,
    chunkslice='times'
)

- Start trajectories simulation - Mon Nov 25 11:31:40 2024
[PID 37590] Diffusion time: 4s 8s 12s 16s 20s 25s 29s 33s 37s 41s 46s 50s 54s 58s 62s 67s 71s 75s 79s 83s 88s 92s 96s 100s 104s 109s 113s 117s 121s 125s 130s 134s 138s 142s 146s 150s 155s 159s 163s 167s 171s 176s 180s 184s 188s 192s 197s 201s 205s 209s 213s 218s 222s 226s 230s 234s 239s 243s 247s 251s 255s 260s 264s 268s 272s 276s 281s 285s 289s 293s 297s 301s 306s 310s 314s 318s 322s 327s 331s 335s 339s 343s 348s 352s 356s 360s 364s 369s 373s 377s 381s 385s 390s 394s 398s 402s 406s 411s 415s 419s 423s 427s 432s 436s 440s 444s 448s 452s 457s 461s 465s 469s 473s 478s 482s 486s 490s 494s 499s 503s
- End trajectories simulation - Mon Nov 25 12:15:22 2024


In [9]:
S.compact_name()

'aa3ff9_P15_D1e-11_86pM_step0.5us_t_max500.0s_ID0-0'

In [10]:
S.store.close()